In [2]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, InputLayer, Dropout, Activation, Flatten, Conv2D
import tensorflow as tf
import numpy as np
import shutil
import random
import cv2
import os



ModuleNotFoundError: No module named 'tensorflow'

In [2]:
# creating train / val /test
root_dir = 'cell_images/'
new_root = 'AllDatasets/'
classes = ['Parasitized', 'Uninfected']

for cls in classes:
    os.makedirs(root_dir + new_root+ 'train/' + cls)
    os.makedirs(root_dir +new_root +'val/' + cls)
    os.makedirs(root_dir +new_root + 'test/' + cls)
    
## creating partition of the data after shuffeling

for cls in classes:
    src = root_dir + cls # folder to copy images from
    print(src)

    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)

    ## here 0.75 = training ratio , (0.95-0.75) = validation ratio , (1-0.95) =  
    ##training ratio  
    train_FileNames,val_FileNames,test_FileNames = np.split(np.array(allFileNames),[int(len(allFileNames)*0.75),int(len(allFileNames)*0.95)])

    # #Converting file names from array to list

    train_FileNames = [src+'/'+ name for name in train_FileNames]
    val_FileNames = [src+'/' + name for name in val_FileNames]
    test_FileNames = [src+'/' + name for name in test_FileNames]

    print('Total images  : '+ cls + ' ' +str(len(allFileNames)))
    print('Training : '+ cls + ' '+str(len(train_FileNames)))
    print('Validation : '+ cls + ' ' +str(len(val_FileNames)))
    print('Testing : '+ cls + ' '+str(len(test_FileNames)))
    
    ## Copy pasting images to target directory

    for name in train_FileNames:
        shutil.copy(name, root_dir + new_root+'train/'+cls )


    for name in val_FileNames:
        shutil.copy(name, root_dir +new_root+'val/'+cls )


    for name in test_FileNames:
        shutil.copy(name,root_dir + new_root+'test/'+cls )

cell_images/Parasitized
Total images  : Parasitized 13780
Training : Parasitized 10335
Validation : Parasitized 2756
Testing : Parasitized 689
cell_images/Uninfected
Total images  : Uninfected 13780
Training : Uninfected 10335
Validation : Uninfected 2756
Testing : Uninfected 689


In [2]:
IMG_WIDTH=128
IMG_HEIGHT=128
img_folder=r'C:\Users\DanilAdmin\malaria\cell_images\AllDatasets\train'

In [1]:
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
                image_path= os.path.join(img_folder, dir1,  file)
                image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
                if image is not None:
                    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
                    image=np.array(image)
                    image = image.astype('float32')
                    image /= 255 
                    img_data_array.append(image)
                    class_name.append(dir1)
    return img_data_array, class_name
# extract the image array and class name
X_train, Y_train =create_dataset(r'C:\Users\DanilAdmin\malaria\cell_images\AllDatasets\train')
X_val, Y_val =create_dataset(r'C:\Users\DanilAdmin\malaria\cell_images\AllDatasets\val')
X_test, Y_test =create_dataset(r'C:\Users\DanilAdmin\malaria\cell_images\AllDatasets\test')

NameError: name 'os' is not defined

In [4]:
target_dict={k: v for v, k in enumerate(np.unique(Y_train))}
target_dict

{'Parasitized': 0, 'Uninfected': 1}

In [5]:
target_val_train=[target_dict[Y_train[i]] for i in range(len(Y_train))]
target_val_val=[target_dict[Y_val[i]] for i in range(len(Y_val))]
target_val_test=[target_dict[Y_val[i]] for i in range(len(Y_val))]

In [6]:
model=tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(IMG_HEIGHT,IMG_WIDTH, 3)),
            tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), padding='same', activation='relu'),
            tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), padding='same', activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(2,activation="softmax")
        ])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
history = model.fit(x=np.array(X_train, np.float32),
                    y=np.array(list(map(int,target_val_train)), np.float32), 
                    batch_size=64, 
                    epochs=10, 
                    validation_data=(np.array(X_val, np.float32), np.array(list(map(int,target_val_val)), np.float32))
                   )

Train on 20668 samples, validate on 5512 samples
Epoch 1/10
20668/20668 [==============================] - 644s 31ms/sample - loss: 0.6463 - acc: 0.6359 - val_loss: 0.5350 - val_acc: 0.7569
Epoch 2/10
20668/20668 [==============================] - 161s 8ms/sample - loss: 0.4807 - acc: 0.8093 - val_loss: 0.3592 - val_acc: 0.8714
Epoch 3/10
20668/20668 [==============================] - 140s 7ms/sample - loss: 0.3902 - acc: 0.8605 - val_loss: 0.3397 - val_acc: 0.8699
Epoch 4/10
20668/20668 [==============================] - 148s 7ms/sample - loss: 0.3386 - acc: 0.8808 - val_loss: 0.3340 - val_acc: 0.8692
Epoch 5/10
20668/20668 [==============================] - 149s 7ms/sample - loss: 0.2783 - acc: 0.9069 - val_loss: 0.2944 - val_acc: 0.8853
Epoch 6/10
20668/20668 [==============================] - 168s 8ms/sample - loss: 0.2104 - acc: 0.9329 - val_loss: 0.2964 - val_acc: 0.8853
Epoch 7/10
20668/20668 [==============================] - 166s 8ms/sample - loss: 0.1518 - acc: 0.9535 - val_l

In [8]:
print("Evaluate on test data")
results = model.evaluate(np.array(X_train, np.float32), np.array(list(map(int,target_val_train)), np.float32), batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
20668/20668 [==============================] - 148s 7ms/sample - loss: 0.0482 - acc: 0.9859
test loss, test acc: [0.04818031876213677, 0.98587185]
